<a href="https://colab.research.google.com/github/albim72/ML_ZAAWANSOWANY_11/blob/main/CNN_obiektowo_full.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
import tensorflow as tf
from tensorflow.keras.layers import Flatten,Dense,Conv2D
from tensorflow.keras import Model

In [27]:
mnist = tf.keras.datasets.mnist

(x_train,y_train),(x_test,y_test) = mnist.load_data()
x_train,x_test = x_train/255.0,x_test/255.0

#dodanie kanałow wymiarów
x_train = x_train[...,tf.newaxis].astype("float32")
x_test = x_test[...,tf.newaxis].astype("float32")

In [28]:
#deklaracja przetwarzania wsadowego i przetasowanie zestawu danych
train_ds = tf.data.Dataset.from_tensor_slices(
    (x_train,y_train)
).shuffle(10000).batch(32)

test_ds = tf.data.Dataset.from_tensor_slices((x_test,y_test)).batch(32)

In [29]:
class MyModel(Model):
  def __init__(self) -> None:
    super(MyModel,self).__init__()
    self.conv1 = Conv2D(32,3,activation='relu')
    self.flatten = Flatten()
    self.d1 = Dense(128,activation='relu')
    self.d2 = Dense(10)

  def __call__(self,x):
    x = self.conv1(x)
    x = self.flatten(x)
    x = self.d1(x)
    return self.d2(x)

model = MyModel()

In [30]:
#wybór optymalizatora i funkcji strat
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.Adam()

In [31]:
#metryki do pomiaru dokładności i straty
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [35]:
@tf.function
def train_step(images,labels):
  with tf.GradientTape() as tape:
    predictions = model(images)
    loss = loss_object(labels,predictions)
  gradients = tape.gradient(loss,model.trainable_variables)
  optimizer.apply_gradients(zip(gradients,model.trainable_variables))

  train_loss(loss)
  train_accuracy(labels,predictions)

In [37]:
@tf.function
def test_step(images,labels):
  predictions = model(images)
  t_loss = loss_object(labels,predictions)

  test_loss(t_loss)
  test_accuracy(labels,predictions)

In [39]:
EPOCHS = 5

for epoch in range(EPOCHS):
  train_loss.reset_states()
  train_accuracy.reset_states()
  test_loss.reset_states()
  test_accuracy.reset_states()

  for images,labels in train_ds:
    train_step(images,labels)

  for test_images,test_labels in test_ds:
    test_step(test_images,test_labels)

  print(
      f'Epoch: {epoch + 1}, '
      f'Loss: {train_loss.result()}, '
      f'Accuracy: {train_accuracy.result()*100}, '
      f'Test Loss: {test_loss.result()}'
      f'Test Accuracy: {test_accuracy.result()*100}, '
  )


Epoch: 1, Loss: 0.02147071249783039Accuracy: 99.29833221435547Test Loss: 0.059468600898981094Test Accuracy: 98.15999603271484
Epoch: 2, Loss: 0.013043652288615704Accuracy: 99.58000183105469Test Loss: 0.06017792969942093Test Accuracy: 98.43000030517578
Epoch: 3, Loss: 0.009314504452049732Accuracy: 99.69499969482422Test Loss: 0.061192795634269714Test Accuracy: 98.54999542236328
Epoch: 4, Loss: 0.006738811731338501Accuracy: 99.76166534423828Test Loss: 0.08883879333734512Test Accuracy: 97.88999938964844
Epoch: 5, Loss: 0.004376511089503765Accuracy: 99.86000061035156Test Loss: 0.08390038460493088Test Accuracy: 98.1500015258789
